In [ ]:
using PandaRobot
using GuSTO

In [ ]:
robot = PandaBot()
env = Factory()
push!(env.obstacle_set, GeometryTypes.HyperRectangle(Vec3f0(0.35,-0.1,0.), Vec3f0(0.2,0.2,0.3)))
# push!(env.obstacle_set, GeometryTypes.HyperRectangle(Vec3f0(0.5,-0.1,0.), Vec3f0(0.2,0.2,0.1)))
# push!(env.obstacle_set, GeometryTypes.HyperRectangle(Vec3f0(0.5,-0.4,0.), Vec3f0(0.2,0.2,0.1)))
# push!(env.obstacle_set, GeometryTypes.HyperRectangle(Vec3f0(0.5,0.2,0.), Vec3f0(0.2,0.2,0.1)))

In [ ]:
# th_init = [0;0;0; -0.1; 0; 0.5; 0]
# th_goal = [0.5;0.5;0.5; -1.; 0; 0.5; 0]
# th_goal = [pi/4; 0; pi/6; 0; pi/4; 0; pi/7]
# th_goal  = [0.5;0.5;0.5; -1.; 0; 0.5; 0]

th_init = [-pi/4;0;0;-0.8*pi;0.25;0.75*pi;0;0;0]
th_init = zeros(robot.num_joints)
th_goal = [pi/4;0;0;-0.8*pi;0.25;0.75*pi;0;0;0]

RigidBodyDynamics.set_configuration!(robot.state, th_goal)
r_EE_inWorldFrame = get_EE_position(robot)

In [ ]:
env = Factory()

model = PandaKin([i for i in r_EE_inWorldFrame])
N = 81
tf_guess = 30.

x_init, x_goal = zeros(model.x_dim), zeros(model.x_dim)
for i in 1:robot.num_joints
    x_init[2*i-1:2*i] = [cos(th_init[i]); sin(th_init[i])]
    x_goal[2*i-1:2*i] = [cos(th_goal[i]); sin(th_goal[i])]
end

In [ ]:
PD = ProblemDefinition(robot, model, env, x_init, x_goal)
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_so1, "Gurobi", OutputFlag=0)

In [ ]:
TOSgusto.SCPS.iter_elapsed_times
TOSgusto.SCPS.iterations
TOSgusto.SCPS.convergence_measure
TOSgusto.SCPS.param.alg.Delta_vec
TOSgusto.SCPS.param.convergence_threshold
TOSgusto.SCPS.param.alg.trust_region_satisfied_vec
TOSgusto.SCPS.param.alg.convex_ineq_satisfied_vec
TOSgusto.SCPS.converged
TOSgusto.SCPS.successful

In [ ]:
joint_traj = zeros(robot.num_joints,N)
for k in 1:N
    joint_traj[:,k] = get_configuration(TOSgusto.traj.X[:,k],model)
end
verify_collision_free(TOSgusto.SCPS.traj, TOSgusto.SCPS.SCPP)
verify_joint_limits(TOSgusto.SCPS.traj, TOSgusto.SCPS.SCPP)

In [ ]:
θf = get_configuration(TOSgusto.traj.X[:,end], model)
RigidBodyDynamics.set_configuration!(robot.state, θf)
r_EE_f = get_EE_position(robot)
println("Distance between r_EE_goal and solution is ", norm(r_EE_f-r_EE_inWorldFrame))
abs.(r_EE_f-r_EE_inWorldFrame)
[r_EE_f r_EE_inWorldFrame]

In [ ]:
using MeshCat
using MeshCatMechanisms

using RigidBodySim
using RigidBodyDynamics

vis = Visualizer()

pd = robot.pan
mvis = MechanismVisualizer(
    pd.mechanism,
    URDFVisuals(PandaRobot.urdfpath(), package_path=[dirname(dirname(PandaRobot.urdfpath()))]),
    vis);

vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(0.,0.,0.,0.5))))
end

q = Vector{Array{Float64,1}}(0)
for k in 1:N
    push!(q, joint_traj[:,k]) 
end

setanimation!(mvis, 1:length(q), q)

plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)
#sleep(1)

In [ ]:
norms = zeros(model.num_joints,N)
for k in 1:N
   for idx in 1:model.num_joints
        norms[idx,k] = sqrt(TOSgusto.traj.X[2*idx-1,k]^2 + TOSgusto.traj.X[2*idx,k]^2)
    end
end

for idx in 1:model.num_joints
    PyPlot.plot(norms[idx,:])
end

In [ ]:
for k in 1:model.num_joints
    PyPlot.plot(joint_traj[k,:])
end